In [11]:
import numpy as np
import math
import os
import random
from matplotlib import pyplot as plt
import queue
import collections
import seaborn as sns
PAGE_SIZE = 4096                    # Page Size in Bytes
NP = 6                              # No. of processes in memory
F = 16                              # No. of Frames in physical memory
INVALID = -10
PID = 0                             # Used for Indexing into Frames
FRAME = 1                           # Used for Indexing into Frames
COMPLETED = 10                      # Process Status Flags
INCOMPLETE = 20                     #
# The following are page replacement algorithm types
FIFO_GLOBAL = 10
oldest_frame = 0
loc_oldest_frame = np.zeros(F).astype('int32')
phys_mem = np.full((F, 2), INVALID).astype('int32')

In [12]:
def init_proc_queue():
    pq = queue.Queue(maxsize=NP)
    proc_queue = list(set((NP * np.random.random(20)).astype('int32')))
    random.shuffle(proc_queue)
    print(proc_queue)
    for pr in proc_queue:
        pq.put_nowait(pr)
    return(pq)

def init_phys_mem():
    for i in range(F):
        #         initializing Physical memory to implement FIFO
        phys_mem[i, PID] = INVALID
        phys_mem[i, FRAME] = INVALID


def update_phys_mem(pid):
    for i in range(F):
        #         Updating physical memory of FIFO
        if phys_mem[i, PID] == pid:
            phys_mem[i, PID] = INVALID
            phys_mem[i, FRAME] = INVALID

In [13]:
def correct_refs(alist, cent, maxpg):
    if cent != 0:
        pgnum = int(cent / PAGE_SIZE)
        lh = (pgnum + 1) * PAGE_SIZE - 1
        ll = pgnum * PAGE_SIZE
    else:
        lh = maxpg * PAGE_SIZE
        ll = 0
    for i in range(alist.shape[0]):
        if alist[i] >= lh:
            alist[i] = lh - 1
        if alist[i] < ll:
            alist[i] = ll + 1
    return alist

def sim_process(M, sigma, outfile, lfrac=0.9):
    cur_page = int(M * np.random.random())  # Generate a random start page
    cur_locale = round(PAGE_SIZE * (cur_page + 0.5))  # Locality of reference
    sum_refs = 0
    list_locales = list()
    for count in range(M):
        # random page address in memory.
        num_refs = int(512 + (PAGE_SIZE - 512) * np.random.random())
        sum_refs += num_refs
        list_locales.append(cur_page)
        num_loc = round(lfrac * num_refs)
        aref_list = np.random.normal(cur_locale, sigma * 300 + 100,
                                     num_loc)  # If siagma is less, it stays less within that locality of reference
        # if sigma more, it may vary more
        aref_list = correct_refs(aref_list, cur_locale, M)
        aref_list = aref_list.astype('int32')
        cur_page = int(M * np.random.random())
        cur_locale = round(PAGE_SIZE * (cur_page + 0.5))
        num_trans = round((1 - lfrac) * num_refs)
        tref_list = np.random.random(num_trans) * cur_locale
        tref_list = correct_refs(tref_list, 0, M)
        tref_list = tref_list.astype('int32')
        np.append(aref_list, tref_list).tofile(outfile, sep='\n')
        outfile.write('\n')

    return list_locales, sum_refs

In [14]:
def get_oldest_frame(pr, algo_type):
    global oldest_frame
    if algo_type == FIFO_GLOBAL:
        rv = oldest_frame
        oldest_frame += 1
        if oldest_frame == F:
            oldest_frame = 0
        return rv

    return INVALID

def getFifoFrame(fr_num, pnum, pr):
    while fr_num < F:
        if phys_mem[fr_num, PID] == pr and phys_mem[fr_num, FRAME] == pnum:
            break
        else:
            fr_num += 1
    return fr_num


def getLruFrame(fr_num, pnum, pr):
    global phys_mem
    while fr_num < F:
        if phys_mem[fr_num, PID] == pr and phys_mem[fr_num, FRAME] == pnum:
            temp = phys_mem[fr_num]
            phys_mem = np.delete(phys_mem, fr_num, axis=0)
            phys_mem = np.concatenate([phys_mem, temp.reshape(1, -1)], axis=0)
            break
        else:
            fr_num += 1
    return fr_num


def getMRUFrame(fr_num, pnum, pr):
    pass


def getRRFrame(fr_num, pnum, pr):
    pass


def getLFUFrame(fr_num, pnum, pr):
    pass

In [16]:
def gen_pagefault(pgnum, pr):
    fr_num = 0
    while fr_num < F and phys_mem[fr_num, FRAME] >= 0:
        fr_num += 1
    if fr_num >= F:
        return INVALID
    phys_mem[fr_num, FRAME] = pgnum
    phys_mem[fr_num, PID] = pr
    return fr_num

In [15]:
def page_replace(pgnum, pr):
    global phys_mem
    temp = np.array([pr, pgnum])
    phys_mem = np.delete(phys_mem, 0, axis=0)
    phys_mem = np.concatenate([phys_mem, temp.reshape(1, -1)], axis=0)
    return F-1


def page_replaceFIFO(pgnum, pr, algo=FIFO_GLOBAL):
    cur_frame = get_oldest_frame(pr, algo)
    phys_mem[cur_frame, PID] = pr
    phys_mem[cur_frame, FRAME] = pgnum
    return cur_frame

In [17]:
def get_pageframe(pnum, pr, algo=None):

    # Trying to get a page Hit below
    fr_num = 0
    if algo == "lru":
        fr_num = getLruFrame(fr_num, pnum, pr)
    elif algo == "fifo":
        fr_num = getFifoFrame(fr_num, pnum, pr)
    elif algo == "mru":
        fr_num = getLruFrame(fr_num, pnum, pr)
    elif algo == "rr":
        fr_num = getRRFrame(fr_num, pnum, pr)
    elif algo == "lfu":
        fr_num = getLFUFrame(fr_num, pnum, pr)

    # This indicates Page Hit.
    if fr_num < F:
        return fr_num * PAGE_SIZE, 0

    # This tries to find page fault if occured.
    fr_num = gen_pagefault(pnum, pr)

    if fr_num >= 0:
        return fr_num * PAGE_SIZE, 1

    # This is for page replacement based on algorith choosen
    if algo == "lru":
        fr_num = page_replace(pnum, pr)
    elif algo == "fifo":
        fr_num = page_replaceFIFO(pnum, pr)

    if fr_num >= 0:
        return fr_num * PAGE_SIZE, 2
    return INVALID, 0